### 재료 1 - 군집화 낫배드

In [14]:
from sklearn.cluster import KMeans # KMeans 라이브러리 불러오기
import numpy as np
import pandas as pd
import math
import seaborn as sb # 데이터 시각화를 위한 seaborn
import matplotlib.pyplot as plt # 데이터 시각화를 위한 matplotlib
import scipy as sp
from scipy.spatial import distance
sb.set_palette("Set2")

In [15]:
from google.cloud import bigquery
import pandas as pd

# BigQuery 클라이언트 초기화
client = bigquery.Client(project='jnu-idv-21')

# 쿼리 작성
query = """
SELECT *
FROM `0808.baby_food_tbl2`
"""

# 쿼리 실행 및 결과를 Pandas DataFrame으로 변환
df = client.query(query).to_dataframe()
df

,food_name,ingredient_name,ingredient_index,ingredient_gram,calorie,carbohydrate,protein,fat,fiber,calcium,phosphorus,iron,sodium,potassium,vitamin_A,thiamine,riboflavin,niacin,vitamin_C
0,찹쌀미음,"[쌀, 찹쌀, 백미]","[0, 18, 19]","[-1, -1, 23]",86.000000,18.700001,1.7,0.1,0.1,0.900000,34.700001,0.5,0.700000,43.900002,0.000000,0.0,0.0,0.4,0.0
1,닭가슴살고구마죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 고구마...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 8, 7]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, -1, 20]",117.000000,21.400000,6.2,0.4,0.7,9.800000,63.400002,0.5,16.400000,157.199997,4.800000,0.0,0.0,2.7,5.2
2,닭가슴살당근죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 당근,...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 47, 7]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, -1, 20]",98.199997,16.900000,6.1,0.4,0.8,12.600000,60.000000,0.5,19.200001,143.800003,252.399994,0.0,0.0,2.7,1.4
3,닭가슴살청경채죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 청경채...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 39]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, 15]",93.500000,15.700000,6.1,0.4,0.7,18.500000,58.299999,0.5,15.600000,105.400002,52.799999,0.0,0.0,2.6,7.4
4,찹쌀컬리플라워당근죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 쌀, 찹쌀, 백미, 컬리플라워,...","[0, 1, 2, 3, 4, 5, 0, 18, 19, 67, 47, 7]","[-1, -1, -1, -1, -1, 20, -1, -1, 5, 10, -1, 10]",94.199997,20.700001,2.0,0.1,0.7,9.000000,29.000000,0.5,6.400000,96.000000,125.699997,0.0,0.0,0.5,5.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,닭고기시금치그라탕,"[닭고기, 가슴살, 날것, 시금치, 생것, 노지, 당근, 생것, 양파, 생것, 국내...","[27, 62, 28, 49, 7, 50, 47, 7, 66, 7, 31, 98, ...","[-1, -1, 30, -1, -1, 10, -1, 10, -1, -1, 10, -...",117.599998,4.200000,11.7,6.0,0.8,133.199997,256.600006,0.6,265.100006,223.399994,241.699997,0.1,0.2,3.5,7.9
144,닭죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 분유,...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 63, 64]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, -1, 15]",162.000000,23.700001,8.8,3.2,0.2,97.599998,123.099998,1.4,35.200001,144.300003,87.599998,0.1,0.2,3.1,7.0
145,도미살무른밥,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 돔, 감성돔, 배추, 생것, 양...","[0, 1, 2, 3, 4, 5, 83, 84, 40, 7, 73, 7, 52, 53]","[-1, -1, -1, -1, -1, 30, -1, 30, -1, 10, -1, 1...",155.000000,27.500000,9.2,0.7,3.3,89.000000,128.800003,1.4,338.700012,982.599976,15.600000,0.1,0.2,2.2,5.2
146,달걀콩닭고기수프,"[계란, 전란, 생것, 닭고기, 가슴살, 날것, 당근, 생것, 완두콩, 생것, 옥수...","[41, 61, 7, 27, 62, 28, 47, 7, 17, 7, 15, 16, ...","[-1, -1, 20, -1, -1, 20, -1, 10, -1, 10, -1, -...",129.899994,11.200000,10.5,4.6,1.5,84.300003,145.100006,1.6,55.900002,228.100006,221.600006,0.2,0.2,7.5,5.7


In [16]:
# 데이터 전처리

# ingredient
data_ingredient = df.iloc[:,:4]
data_ingredient

,food_name,ingredient_name,ingredient_index,ingredient_gram
0,찹쌀미음,"[쌀, 찹쌀, 백미]","[0, 18, 19]","[-1, -1, 23]"
1,닭가슴살고구마죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 고구마...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 8, 7]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, -1, 20]"
2,닭가슴살당근죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 당근,...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 47, 7]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, -1, 20]"
3,닭가슴살청경채죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 청경채...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 39]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, 15]"
4,찹쌀컬리플라워당근죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 쌀, 찹쌀, 백미, 컬리플라워,...","[0, 1, 2, 3, 4, 5, 0, 18, 19, 67, 47, 7]","[-1, -1, -1, -1, -1, 20, -1, -1, 5, 10, -1, 10]"
...,...,...,...,...
143,닭고기시금치그라탕,"[닭고기, 가슴살, 날것, 시금치, 생것, 노지, 당근, 생것, 양파, 생것, 국내...","[27, 62, 28, 49, 7, 50, 47, 7, 66, 7, 31, 98, ...","[-1, -1, 30, -1, -1, 10, -1, 10, -1, -1, 10, -..."
144,닭죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 분유,...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 63, 64]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, -1, 15]"
145,도미살무른밥,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 돔, 감성돔, 배추, 생것, 양...","[0, 1, 2, 3, 4, 5, 83, 84, 40, 7, 73, 7, 52, 53]","[-1, -1, -1, -1, -1, 30, -1, 30, -1, 10, -1, 1..."
146,달걀콩닭고기수프,"[계란, 전란, 생것, 닭고기, 가슴살, 날것, 당근, 생것, 완두콩, 생것, 옥수...","[41, 61, 7, 27, 62, 28, 47, 7, 17, 7, 15, 16, ...","[-1, -1, 20, -1, -1, 20, -1, 10, -1, 10, -1, -..."


In [17]:
# 재료 리스트를 공백으로 구분된 하나의 문자열로 변환
df['ingredients_str'] = df['ingredient_name'].apply(lambda x: ' '.join(x))
df['ingredients_str']

0                                                쌀 찹쌀 백미
1                쌀 멥쌀 논벼 백미 국내산 일반형 일품 닭고기 가슴살 날것 고구마 생것
2                 쌀 멥쌀 논벼 백미 국내산 일반형 일품 닭고기 가슴살 날것 당근 생것
3              쌀 멥쌀 논벼 백미 국내산 일반형 일품 닭고기 가슴살 날것 청경채 중국채소
4              쌀 멥쌀 논벼 백미 국내산 일반형 일품 쌀 찹쌀 백미 컬리플라워 당근 생것
                             ...                        
143    닭고기 가슴살 날것 시금치 생것 노지 당근 생것 양파 생것 국내산 우유 가공우유 보...
144             쌀 멥쌀 논벼 백미 국내산 일반형 일품 닭고기 가슴살 날것 분유 조제분유
145     쌀 멥쌀 논벼 백미 국내산 일반형 일품 돔 감성돔 배추 생것 양송이 생것 다시마 말린것
146    계란 전란 생것 닭고기 가슴살 날것 당근 생것 완두콩 생것 옥수수 찰옥수수 생것 분...
147                   쌀 멥쌀 논벼 백미 국내산 일반형 일품 미역 말린것 당근 생것
Name: ingredients_str, Length: 148, dtype: object

In [19]:
# CountVectorizer를 사용하여 특징 벡터 생성
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['ingredients_str'])

In [21]:
# KMeans 군집화 모델 생성
num_clusters = 3  # 군집 개수 설정
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(X)

KMeans(n_clusters=3, random_state=42)

In [23]:
# 군집 결과를 데이터프레임에 추가
df['cluster'] = kmeans.labels_
df

,food_name,ingredient_name,ingredient_index,ingredient_gram,calorie,carbohydrate,protein,fat,fiber,calcium,...,iron,sodium,potassium,vitamin_A,thiamine,riboflavin,niacin,vitamin_C,ingredients_str,cluster
0,찹쌀미음,"[쌀, 찹쌀, 백미]","[0, 18, 19]","[-1, -1, 23]",86.000000,18.700001,1.7,0.1,0.1,0.900000,...,0.5,0.700000,43.900002,0.000000,0.0,0.0,0.4,0.0,쌀 찹쌀 백미,2
1,닭가슴살고구마죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 고구마...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 8, 7]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, -1, 20]",117.000000,21.400000,6.2,0.4,0.7,9.800000,...,0.5,16.400000,157.199997,4.800000,0.0,0.0,2.7,5.2,쌀 멥쌀 논벼 백미 국내산 일반형 일품 닭고기 가슴살 날것 고구마 생것,1
2,닭가슴살당근죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 당근,...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 47, 7]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, -1, 20]",98.199997,16.900000,6.1,0.4,0.8,12.600000,...,0.5,19.200001,143.800003,252.399994,0.0,0.0,2.7,1.4,쌀 멥쌀 논벼 백미 국내산 일반형 일품 닭고기 가슴살 날것 당근 생것,1
3,닭가슴살청경채죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 청경채...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 39]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, 15]",93.500000,15.700000,6.1,0.4,0.7,18.500000,...,0.5,15.600000,105.400002,52.799999,0.0,0.0,2.6,7.4,쌀 멥쌀 논벼 백미 국내산 일반형 일품 닭고기 가슴살 날것 청경채 중국채소,1
4,찹쌀컬리플라워당근죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 쌀, 찹쌀, 백미, 컬리플라워,...","[0, 1, 2, 3, 4, 5, 0, 18, 19, 67, 47, 7]","[-1, -1, -1, -1, -1, 20, -1, -1, 5, 10, -1, 10]",94.199997,20.700001,2.0,0.1,0.7,9.000000,...,0.5,6.400000,96.000000,125.699997,0.0,0.0,0.5,5.4,쌀 멥쌀 논벼 백미 국내산 일반형 일품 쌀 찹쌀 백미 컬리플라워 당근 생것,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,닭고기시금치그라탕,"[닭고기, 가슴살, 날것, 시금치, 생것, 노지, 당근, 생것, 양파, 생것, 국내...","[27, 62, 28, 49, 7, 50, 47, 7, 66, 7, 31, 98, ...","[-1, -1, 30, -1, -1, 10, -1, 10, -1, -1, 10, -...",117.599998,4.200000,11.7,6.0,0.8,133.199997,...,0.6,265.100006,223.399994,241.699997,0.1,0.2,3.5,7.9,닭고기 가슴살 날것 시금치 생것 노지 당근 생것 양파 생것 국내산 우유 가공우유 보...,0
144,닭죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 분유,...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 63, 64]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, -1, 15]",162.000000,23.700001,8.8,3.2,0.2,97.599998,...,1.4,35.200001,144.300003,87.599998,0.1,0.2,3.1,7.0,쌀 멥쌀 논벼 백미 국내산 일반형 일품 닭고기 가슴살 날것 분유 조제분유,1
145,도미살무른밥,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 돔, 감성돔, 배추, 생것, 양...","[0, 1, 2, 3, 4, 5, 83, 84, 40, 7, 73, 7, 52, 53]","[-1, -1, -1, -1, -1, 30, -1, 30, -1, 10, -1, 1...",155.000000,27.500000,9.2,0.7,3.3,89.000000,...,1.4,338.700012,982.599976,15.600000,0.1,0.2,2.2,5.2,쌀 멥쌀 논벼 백미 국내산 일반형 일품 돔 감성돔 배추 생것 양송이 생것 다시마 말린것,1
146,달걀콩닭고기수프,"[계란, 전란, 생것, 닭고기, 가슴살, 날것, 당근, 생것, 완두콩, 생것, 옥수...","[41, 61, 7, 27, 62, 28, 47, 7, 17, 7, 15, 16, ...","[-1, -1, 20, -1, -1, 20, -1, 10, -1, 10, -1, -...",129.899994,11.200000,10.5,4.6,1.5,84.300003,...,1.6,55.900002,228.100006,221.600006,0.2,0.2,7.5,5.7,계란 전란 생것 닭고기 가슴살 날것 당근 생것 완두콩 생것 옥수수 찰옥수수 생것 분...,0


In [24]:
# 군집별 대표 재료 찾기
cluster_centers = kmeans.cluster_centers_
top_words = vectorizer.get_feature_names_out()
for i, center in enumerate(cluster_centers):
    top_indices = center.argsort()[-5:]  # 각 군집에서 가장 높은 빈도의 단어 인덱스 선택
    top_ingredients = [top_words[idx] for idx in top_indices]
    print(f"Cluster {i+1} Top Ingredients: {', '.join(top_ingredients)}")

Cluster 1 Top Ingredients: 멥쌀, 계란, 당근, 국내산, 생것
Cluster 2 Top Ingredients: 논벼, 백미, 일반형, 국내산, 멥쌀
Cluster 3 Top Ingredients: 조제분유, 부사, 사과, 생것, 생과


In [30]:
cluster1 = df[df['cluster'] == 1]
cluster1

,food_name,ingredient_name,ingredient_index,ingredient_gram,calorie,carbohydrate,protein,fat,fiber,calcium,...,iron,sodium,potassium,vitamin_A,thiamine,riboflavin,niacin,vitamin_C,ingredients_str,cluster
1,닭가슴살고구마죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 고구마...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 8, 7]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, -1, 20]",117.000000,21.400000,6.2,0.4,0.7,9.800000,...,0.5,16.400000,157.199997,4.800000,0.0,0.0,2.7,5.2,쌀 멥쌀 논벼 백미 국내산 일반형 일품 닭고기 가슴살 날것 고구마 생것,1
2,닭가슴살당근죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 당근,...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 47, 7]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, -1, 20]",98.199997,16.900000,6.1,0.4,0.8,12.600000,...,0.5,19.200001,143.800003,252.399994,0.0,0.0,2.7,1.4,쌀 멥쌀 논벼 백미 국내산 일반형 일품 닭고기 가슴살 날것 당근 생것,1
3,닭가슴살청경채죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 청경채...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 39]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, 15]",93.500000,15.700000,6.1,0.4,0.7,18.500000,...,0.5,15.600000,105.400002,52.799999,0.0,0.0,2.6,7.4,쌀 멥쌀 논벼 백미 국내산 일반형 일품 닭고기 가슴살 날것 청경채 중국채소,1
4,찹쌀컬리플라워당근죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 쌀, 찹쌀, 백미, 컬리플라워,...","[0, 1, 2, 3, 4, 5, 0, 18, 19, 67, 47, 7]","[-1, -1, -1, -1, -1, 20, -1, -1, 5, 10, -1, 10]",94.199997,20.700001,2.0,0.1,0.7,9.000000,...,0.5,6.400000,96.000000,125.699997,0.0,0.0,0.5,5.4,쌀 멥쌀 논벼 백미 국내산 일반형 일품 쌀 찹쌀 백미 컬리플라워 당근 생것,1
7,애호박완두콩무른밥,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 호박, 애호박, 생것, 완두콩,...","[0, 1, 2, 3, 4, 5, 13, 14, 7, 17, 7]","[-1, -1, -1, -1, -1, 25, -1, -1, 20, -1, 5]",100.599998,22.000000,2.3,0.2,0.8,9.900000,...,0.5,1.000000,98.199997,8.000000,0.1,0.0,2.7,7.0,쌀 멥쌀 논벼 백미 국내산 일반형 일품 호박 애호박 생것 완두콩 생것,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,연두부아욱진밥,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 두부, 연두부, 아욱]","[0, 1, 2, 3, 4, 5, 46, 51, 68]","[-1, -1, -1, -1, -1, 30, -1, 30, 25]",121.699997,23.700001,4.4,1.0,1.5,46.299999,...,1.3,10.800000,195.600006,285.799988,0.1,0.1,0.7,12.0,쌀 멥쌀 논벼 백미 국내산 일반형 일품 두부 연두부 아욱,1
138,밥경단,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 쇠고기, 한우, 사태, 표고버섯...","[0, 1, 2, 3, 4, 5, 21, 22, 23, 77, 7, 68]","[-1, -1, -1, -1, -1, 30, -1, -1, 20, -1, 15, 15]",140.100006,24.100000,7.7,12.9,1.8,20.500000,...,1.3,17.600000,197.500000,173.800003,0.1,0.1,2.2,8.4,쌀 멥쌀 논벼 백미 국내산 일반형 일품 쇠고기 한우 사태 표고버섯 생것 아욱,1
144,닭죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 분유,...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 63, 64]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, -1, 15]",162.000000,23.700001,8.8,3.2,0.2,97.599998,...,1.4,35.200001,144.300003,87.599998,0.1,0.2,3.1,7.0,쌀 멥쌀 논벼 백미 국내산 일반형 일품 닭고기 가슴살 날것 분유 조제분유,1
145,도미살무른밥,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 돔, 감성돔, 배추, 생것, 양...","[0, 1, 2, 3, 4, 5, 83, 84, 40, 7, 73, 7, 52, 53]","[-1, -1, -1, -1, -1, 30, -1, 30, -1, 10, -1, 1...",155.000000,27.500000,9.2,0.7,3.3,89.000000,...,1.4,338.700012,982.599976,15.600000,0.1,0.2,2.2,5.2,쌀 멥쌀 논벼 백미 국내산 일반형 일품 돔 감성돔 배추 생것 양송이 생것 다시마 말린것,1


In [31]:
cluster1 = df[df['cluster'] == 0]
cluster1

,food_name,ingredient_name,ingredient_index,ingredient_gram,calorie,carbohydrate,protein,fat,fiber,calcium,...,iron,sodium,potassium,vitamin_A,thiamine,riboflavin,niacin,vitamin_C,ingredients_str,cluster
28,고구마야채무른밥,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 고구마, 생것, 양배추, 생것,...","[0, 1, 2, 3, 4, 5, 8, 7, 36, 7, 47, 7, 9, 7]","[-1, -1, -1, -1, -1, 25, -1, 5, -1, 10, -1, 10...",102.699997,22.600000,2.5,0.2,1.2,18.700001,...,0.6,5.600000,136.000000,139.800003,0.0,0.0,0.6,14.600000,쌀 멥쌀 논벼 백미 국내산 일반형 일품 고구마 생것 양배추 생것 당근 생것 브로콜리 생것,0
81,닭고기완자핑거푸드,"[닭고기, 날것, 양파, 생것, 국내산, 우엉, 생것, 당근, 생것]","[27, 28, 66, 7, 31, 82, 7, 47, 7]","[-1, 30, -1, -1, 10, -1, 10, -1, 10]",65.000000,3.000000,6.1,3.2,0.9,14.200000,...,0.5,21.000000,141.300003,142.199997,0.0,0.1,2.3,2.300000,닭고기 날것 양파 생것 국내산 우엉 생것 당근 생것,0
88,표고버섯달걀노른자죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 표고버섯, 생것, 배추, 생것,...","[0, 1, 2, 3, 4, 5, 77, 7, 40, 7, 41, 42, 7]","[-1, -1, -1, -1, -1, 20, -1, 5, -1, 10, -1, -1...",140.500000,16.000000,4.7,6.0,0.6,32.599998,...,1.5,10.200000,73.400002,91.199997,0.1,0.1,0.6,3.100000,쌀 멥쌀 논벼 백미 국내산 일반형 일품 표고버섯 생것 배추 생것 계란 난황 생것,0
94,치즈야채무른밥,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 당근, 생것, 호박, 애호박, ...","[0, 1, 2, 3, 4, 5, 47, 7, 13, 14, 7, 66, 7, 31...","[-1, -1, -1, -1, -1, 30, -1, 10, -1, -1, 10, -...",161.899994,26.400000,5.0,3.8,0.8,86.900002,...,0.6,174.300003,121.699997,164.100006,0.1,0.1,0.6,4.900000,쌀 멥쌀 논벼 백미 국내산 일반형 일품 당근 생것 호박 애호박 생것 양파 생것 국내...,0
97,감자야채달걀노른자찜,"[감자, 생것, 배추, 생것, 계란, 난황, 생것]","[6, 7, 40, 7, 41, 42, 7]","[-1, 10, -1, 10, -1, -1, 20]",74.500000,1.600000,3.5,6.0,0.3,30.200001,...,1.2,9.900000,79.800003,91.199997,0.1,0.1,0.2,4.900000,감자 생것 배추 생것 계란 난황 생것,0
113,명태옥수수달걀진밥,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 명태, 생것, 옥수수, 찰옥수수...","[0, 1, 2, 3, 4, 5, 87, 7, 15, 16, 7, 39, 41, 4...","[-1, -1, -1, -1, -1, 20, -1, 10, -1, -1, 5, 10...",165.000000,19.299999,7.2,6.3,2.1,77.800003,...,1.9,181.800003,486.600006,133.399994,0.1,0.1,0.9,6.000000,쌀 멥쌀 논벼 백미 국내산 일반형 일품 명태 생것 옥수수 찰옥수수 생것 청경채 중국...,0
115,닭가슴살요구르트샐러드,"[닭고기, 가슴살, 날것, 양배추, 생것, 감자, 생것, 당근, 생것, 요구르트, 호상]","[27, 62, 28, 36, 7, 6, 7, 47, 7, 108, 109]","[-1, -1, 30, -1, 10, -1, 10, -1, 10, -1, 10]",55.000000,4.400000,7.8,0.6,0.7,28.100000,...,0.4,23.500000,174.500000,130.399994,0.1,0.1,3.6,6.700000,닭고기 가슴살 날것 양배추 생것 감자 생것 당근 생것 요구르트 호상,0
121,버섯덮밥,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 표고버섯, 생것, 양송이, 생것...","[0, 1, 2, 3, 4, 5, 77, 7, 73, 7, 39, 66, 7, 31...","[-1, -1, -1, -1, -1, 30, -1, 5, -1, 5, 10, -1,...",145.199997,32.599998,2.5,0.2,1.1,17.400000,...,0.8,3.400000,110.900002,34.500000,0.0,0.1,0.9,6.000000,쌀 멥쌀 논벼 백미 국내산 일반형 일품 표고버섯 생것 양송이 생것 청경채 중국채소 ...,0
122,감자완자핑거푸드,"[감자, 생것, 쇠고기, 한우, 사태, 당근, 생것, 브로콜리, 생것]","[6, 7, 21, 22, 23, 47, 7, 9, 7]","[-1, 30, -1, -1, 10, -1, 15, -1, 15]",39.299999,5.200000,4.1,6.4,1.3,17.900000,...,0.8,12.400000,242.399994,209.000000,0.1,0.1,1.2,22.100000,감자 생것 쇠고기 한우 사태 당근 생것 브로콜리 생것,0
125,우엉오므라이스,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 날것, 우엉, 생것, ...","[0, 1, 2, 3, 4, 5, 27, 28, 82, 7, 47, 7, 41, 4...","[-1, -1, -1, -1, -1, 20, -1, 10, -1, 15, -1, 1...",152.899994,18.700001,5.3,6.2,1.3,30.900000,...,1.1,15.600000,156.800003,239.399994,0.1,0.1,1.2,1.500000,쌀 멥쌀 논벼 백미 국내산 일반형 일품 닭고기 날것 우엉 생것 당근 생것 계란 난황...,0


In [32]:
cluster1 = df[df['cluster'] == 2]
cluster1

,food_name,ingredient_name,ingredient_index,ingredient_gram,calorie,carbohydrate,protein,fat,fiber,calcium,...,iron,sodium,potassium,vitamin_A,thiamine,riboflavin,niacin,vitamin_C,ingredients_str,cluster
0,찹쌀미음,"[쌀, 찹쌀, 백미]","[0, 18, 19]","[-1, -1, 23]",86.000000,18.700001,1.7,0.1,0.1,0.900000,...,0.5,0.7,43.900002,0.000000,0.0,0.0,0.4,0.000000,쌀 찹쌀 백미,2
5,고구마찹쌀진밥,"[쌀, 찹쌀, 백미, 고구마, 생것]","[0, 18, 19, 8, 7]","[-1, -1, 20, -1, 10]",87.599998,19.299999,1.6,0.1,0.4,3.200000,...,0.5,2.1,81.099998,1.900000,0.0,0.0,0.4,2.500000,쌀 찹쌀 백미 고구마 생것,2
6,멜론감자수프,"[멜론, 머스크, 감자, 생것]","[85, 86, 6, 7]","[-1, 50, -1, 30]",35.500000,8.000000,1.4,0.1,0.9,5.300000,...,0.5,7.4,305.799988,1.500000,0.1,0.0,0.8,17.299999,멜론 머스크 감자 생것,2
8,감자바나나요구르트으깸,"[감자, 생것, 바나나, 생과, 요구르트, 호상]","[6, 7, 29, 11, 108, 109]","[-1, 30, -1, 30, -1, 10]",54.700001,12.400000,1.5,0.3,1.0,20.500000,...,0.5,1.8,219.300003,5.000000,0.0,0.0,0.6,9.500000,감자 생것 바나나 생과 요구르트 호상,2
21,사과퓨레,"[사과, 생과, 부사]","[10, 11, 12]","[-1, -1, 100]",57.000000,15.300000,0.3,0.1,0.0,3.000000,...,0.3,3.0,95.000000,3.000000,0.0,0.0,0.1,4.000000,사과 생과 부사,2
25,고구마사과조림,"[고구마, 생것, 사과, 생과, 부사, 전분, 감자녹말]","[8, 7, 10, 11, 12, 43, 44]","[-1, 20, -1, -1, 30, -1, 20]",109.099998,27.000000,0.4,0.1,0.5,10.100000,...,0.6,4.9,120.699997,4.700000,0.0,0.0,0.2,6.200000,고구마 생것 사과 생과 부사 전분 감자녹말,2
29,당근퓨레,"[당근, 생것, 바나나, 생과]","[47, 7, 29, 11]","[-1, 50, -1, 50]",63.500000,16.000000,1.1,0.2,2.4,22.500000,...,0.6,15.5,348.500000,632.000000,0.0,0.0,0.6,7.000000,당근 생것 바나나 생과,2
30,바나나퓨레,"[바나나, 생과]","[29, 11]","[-1, 100]",93.000000,24.100000,1.2,0.2,1.8,7.000000,...,0.6,2.0,335.000000,7.000000,0.0,0.0,0.5,8.000000,바나나 생과,2
37,오렌지퓨레,"[오렌지, 생과]","[110, 11]","[-1, 100]",40.000000,9.900000,0.8,0.2,1.9,39.000000,...,0.1,5.0,126.000000,13.000000,0.1,0.0,0.4,46.000000,오렌지 생과,2
52,과일요구르트범벅,"[바나나, 생과, 사과, 생과, 부사, 오렌지, 생과, 요구르트, 호상]","[29, 11, 10, 11, 12, 110, 11, 108, 109]","[-1, 20, -1, -1, 20, -1, 10, -1, 10]",44.299999,10.600000,0.7,0.3,0.6,22.500000,...,0.2,1.8,98.599998,6.200000,0.0,0.0,0.2,7.800000,바나나 생과 사과 생과 부사 오렌지 생과 요구르트 호상,2


In [ ]:
# 결론
# cluster0 = 엄청엄청 어린 아이들이 먹는 거
# cluster1 = 좀 자라서 유치원생도 먹을 수 있는 진짜 음식 같은 거
# cluster 2 = 디저트, 주스 등등